In [ ]:
%matplotlib inline
import requests
import bs4
import numpy as np
import pandas as pd

# Parsing Nested Data

## Last time: scraping web data

* GET requests result in HTML; What does this look like?
* Let's see what is on the UCSD Data Science Events page.

In [ ]:
url = "http://dsc.ucsd.edu/node/10"

r = requests.get(url)
    
urlText = r.text

Nchars = 10000
print(urlText[:Nchars]) # Print the first 500 characters
print("\n\n... " + str(len(urlText)-Nchars) + " additional characters")


In [ ]:
len(r.text)

## Extracting information from HTML

<div class="image-txt-container">

* If you want to scrape tabular data:
    - it's as easy as `pd.read_html`

* Scraping `basketball-reference.com` tables:

<img src="imgs/nba.png" width="50%">
</div>

In [ ]:
nba = pd.read_html('https://www.basketball-reference.com/leagues/NBA_2017_per_game.html')

In [ ]:
nba[0].head()

## Extracting information from HTML

<div class="image-txt-container">

* If your data is *not* tabular:
    - need to parse the HTML
    - determine what to extract
    - transform HTML to a tabular structure
    
* The data may lies on multiple pages.

<img src="imgs/dsc.png" width="50%">
</div>


In [ ]:
Nchars = 10000
print(urlText[:Nchars]) # Print the first 500 characters
print("\n\n... " + str(len(urlText)-Nchars) + " additional characters")

## What is HTML?

* HTML (HyperText Markup Language) is the most basic building block of the Web. 
* It defines the content and layout of a web-page.

* HTML markup includes special "elements" (tags) such as 
    * `<head>, <title>, <body>, <p>, <div>, <img>`,.....
    

See [this tutorial](http://fab.academany.org/2018/labs/fablaboshanghai/students/bob-wu/Fabclass/week2_project_management/HTML.html) for more reference.

In [ ]:
!cat data/lec10.html

In [ ]:
from IPython.display import HTML

In [ ]:
# Display an HTML page, inline
HTML(open('data/lec10.html').read())

### The Anatomy of HTML

* **HTML Document**: the totality of markup that makes up a web-page
* **Document Object Model**: the internal representation of a HTML document as a *tree* structure.

* **HTML Element**: An object in the DOM, such as a paragraph, header, title.
* **HTML Tags**: Markers that denote the *start* and *end* of an element. E.g. `<p>` and `</p>`.

In [ ]:
!cat data/lec10.html

### HTML Tags

HTML tags define both:
* document structure elements and 
* document head/body elements.

<img src="imgs/webpage_anatomy.png" width="50%">

### Useful tags to know:

|Structure Elements|Description|Head/Body Elements|Description|
|---|---|---|---|
|`<html>`|the document|`<p>`|the paragraph|
|`<head>`|the header|`<h1>, <h2>, ...`|header(s)|
|`<body>`|the body|`<img>`|images|
|`<div>` |a logical division of the document|`<a>`| anchor (hyper-link)|
|`<span>`|an *in-line* logical division|[MANY MORE](https://en.wikipedia.org/wiki/HTML_element)||


### Example: Images and Hyperlinks

* Tag for a picture (can use a link to the image):
```
<img src="HumDum.png" alt="Humbpty Dumpty">
```

* Tag for a hyperlink: 

```
<a href="https://ucsd.edu/">Visit our page!</a>
```


In [ ]:
!cat data/lec10_pic_ref.html

In [ ]:
HTML(open('data/lec10_pic_ref.html').read())

# The HTML Document Tree

## The HTML Document Tree

* DOM represents a document as a logical tree.

<div class="image-txt-container">

<img src="imgs/webpage_anatomy.png" width="50%">

<img src="imgs/dom_tree.png" width="50%">

</div>    

### The tree structure *does* matter

* HTML parsing methods return tree-like objects.
* Tree algorithms used (e.g. BFS vs DFS) affect parsing performance.
* "Flattening" trees to tables requires choices (and DIY techniques).
* Manipulation of pages (e.g. with javascript) requires manipulating tree structures.

### Question: "quotes collection" website

* What do you think the DOM tree look like? (roughly)
* What would your table schema (i.e. rows/columns) look like?

<img src="imgs/quotes2scrape.png">

### Example (rough) document tree

* How you would you parse it, if you wanted to collect data:
    - Quote-by-quote (all attributes)?
    - Attribute-by-attribute?
    
<div class="image-txt-container">

<img src="imgs/quotes2scrape.png" width="50%">
    
<img src="imgs/quote_dom.png" width="50%">

</div> 

## BeautifulSoup: parsing the document tree

* [Beautiful Soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) is a python HTML parser.
* **Warning:** BeautifulSoup has changed between versions, so make sure you are looking at documentation for the version you are using (4 here).

* Parse a small HTML "page", with corresponding tree below:

<img src="imgs/dom_tree_1.png" width="75%">

In [ ]:
s = '''
<body>

  <div id="content">
    <h1>Heading here</h1>
    <p>My First paragraph</p>
    <p>My <em>second</em> paragraph</p>
    <hr>
  </div>
  
  <div id="nav">
    <ul>
      <li>item 1</li>
      <li>item 2</li>
      <li>item 3</li>
    </ul>
  </div>

</body>
'''

In [ ]:
HTML(s)

### BeautifulSoup Parsing

* `bs4.BeautifulSoup` parses a string or file-like object representing HTML
* Returns a *parsed document*
* Use the `children` property to access child nodes.

In [ ]:
# help(bs4.BeautifulSoup)

In [ ]:
soup = bs4.BeautifulSoup(s)
soup

In [ ]:
type(soup)

In [ ]:
# print just the text
print(soup.text)

In [ ]:
soup.children
list(soup.children)

In [ ]:
root = list(soup.children)[0]

list(root.children)

### Discussion Question

What is the output of the following code:
```
list(list(list(soup.children)[0].children)[0].children)[-2]
```
(*Note:* the output is a node element; if `-2` were `-1`, then the value would be `\n`)


<img src="imgs/dom_tree_1.png" width="75%">

In [ ]:
list(list(list(soup.children)[0].children)[0].children)[-2]

In [ ]:
list(list(list(soup.children)[0].children)[0].children)

### Document tree traversal: depth-first

* Using `.children` attribute, traverse `soup` depth-first.
* Take care to only print node elements!

<img src="imgs/dom_tree_1.png" width="75%">

In [ ]:
soup

### Document tree traversal: depth-first

* Using `.children` attribute, traverse `soup` depth-first.
* Take care to only print node elements!

<img src="imgs/dom_tree_1.png" width="75%">

In [ ]:
# depth first search

def dfs(elt, visited):
    if elt not in visited:
        visited.append(elt)
        print(elt.name)
        for e in elt:
            if not isinstance(e, str):
                dfs(e, visited)
    return visited

In [ ]:
dfs(soup, [])

In [ ]:
# DFS using `descendants` property
children = []
for x in soup.descendants:
    children.append(x)

In [ ]:
children

## Selecting attributes of nodes
* The `.text` property of a tag element gets the text elements between the tags.
* The `.attrs` property lists all attributes of a tag.
* The `.get(key)` method, gets the value of a tag attribute.

In [ ]:
hdr = list(soup.descendants)[5]
hdr

In [ ]:
hdr.text

In [ ]:
div = list(soup.descendants)[20]
div

In [ ]:
div = list(soup.descendants)[20]
div

In [ ]:
div.attrs

In [ ]:
div.get('id')

## Selecting subtrees of the document tree

* Use the BeautifulSoup methods `find_**` to select subtrees
* `soup.find(name=None, attrs={}, recursive=True, text=None, **kwargs)`

## Selecting subtrees of the document tree

* Using `soup.find('div')`:


<div class="image-txt-container">
    
<img src="imgs/dom_tree_1.png" width="50%">
  
<img src="imgs/dom_subtree_1.png" width="40%">
    
</div>

In [ ]:
div = soup.find('div')
div

In [ ]:
type(div)

In [ ]:
soup.find('div', attrs={'id': 'nav'})

In [ ]:
soup.find_all('div')

In [ ]:
[x.text for x in soup.find_all('li')]

### Example: extracting data from HTML

* Let's parse the `dsc.ucsd.edu` schedule

In [ ]:
# url = "http://dsc.ucsd.edu/node/10"
# r = requests.get(url)   
# urlText = r.text


from bs4 import BeautifulSoup
soup = BeautifulSoup(urlText, 'html.parser')
#soup

In [ ]:
# we can extract the title of the document

soup.find('title')
# soup.title.text

In [ ]:
soup.find('title').contents

In [ ]:
# we can extract the first paragraph 

soup.find('p')

# open link in the browser, right click and "page source". Can you find <p> tags?
# and hyperlinks

In [ ]:
# Grab all the links

all_links  = soup.find_all('a')
all_links


In [ ]:
link = all_links[10]

In [ ]:
link.attrs

In [ ]:
link.get('href')

In [ ]:
# print all the links and their urls
for link in soup.find_all('a'):
    print('%s:%s(%s)' %(link.text, ' '*(50 - len(link.text)) , link.get('href', ' ')[:50]))


In [ ]:
# Show all the text as a string
print(soup.get_text())

### Example: scraping quotes
* Collect famous quotes and related data
* Requires scraping many pages to get data
* Parse the pages to extract information

<img src="imgs/quotes2scrape.png" width="50%">

### Scraping quotes: steps
1. What information to collect?
2. What pages are needed to visit? In what order?
3. What does our table schema look like?

<img src="imgs/quotes2scrape.png" width="50%">

In [ ]:
# scraping quotes:
# Columns: quote, author, author bio, binary column for each tag
# Create a dictionary (JSON) for each record
# Translate dictionary => dataframe
# make 1-2 observations about the data.

### Scraping quotes: proposed plan

1. Function `scrape_pages(urllist)` that scrapes a list of urls.
    - `requests` is only used here!    
2. Function `parse_quote_list(soup)` that selects the quotes information from a quote page.
3. Function `parse_author_info(author_soup)` that selects the author information from an author page.

**Tip:** Have functions that request and functions that parse, but not both! 
- Easier to debug and catch errors!
- Avoids unnecessary requests!

In [ ]:
# proposed plan:

def scrape_pages(urllist):
    
    out = []
    for url in urllist:
        
        # 1. request pages
        # 2. parse page w/bs4
        # 3. parse each quote on quote page
        #         4. extract info from each quote
        #         5. get author URL for each quote
        #         6. request auth page / parse / extract
        # 7. Add parsed info (DF) to out
        pass
        
    return pd.concat(out)

### Scraping quotes: `parse_quote_list(soup)`
* Develop function on a single page of quotes
* Wrap code in `parse_quote_list`

In [ ]:
resp = requests.get('http://quotes.toscrape.com/page/%d/' % 1)
soup = bs4.BeautifulSoup(resp.text)

In [ ]:
# Need: quote text, author name, author url, tags
soup

In [ ]:
# grab each quote and parse!
# A quote is defined by a div with class "quote"!
quote = soup.find('div', attrs={'class': 'quote'})
quote

In [ ]:
# quote text
quote_text = quote.find('span', attrs='text').text
quote_text

In [ ]:
# author name
quote_auth = quote.find('small', attrs={'class': 'author'}).text
quote_auth

In [ ]:
# author url
auth_url = quote.find('a').get('href')
auth_url = 'http://quotes.toscrape.com' + auth_url
auth_url

In [ ]:
# tags
quote.find('div', attrs={'class': 'tags'})

#quote.find('div', attrs={'class': 'tags'}).find_all('a')

#quote_tags = [x.text for x in quote.find('div', attrs={'class': 'tags'}).find_all('a')]
#quote_tags

In [ ]:
row = {}
row['quote_text'] = quote_text
row['quote_auth'] = quote_auth
row['auth_url'] = auth_url
for tag in quote_tags:
    row[tag] = 1

# Single row
pd.Series(row)

In [ ]:
def parse_quote_list(soup):
    
    out = []
    
    # Loop through all quotes
    quotes = soup.find_all('div', attrs={'class': 'quote'})
    for quote in quotes:
        
        # parse quote elements
        quote_text = quote.find('span', attrs='text').text
        quote_auth = quote.find('small', attrs={'class': 'author'}).text
        auth_url = quote.find('a').get('href')
        quote_tags = [x.text for x in quote.find('div', attrs={'class': 'tags'}).find_all('a')]
        
        # column - value mapping
        row = {}
        row['quote_text'] = quote_text
        row['quote_auth'] = quote_auth
        row['auth_url'] = 'http://quotes.toscrape.com' + auth_url
        for tag in quote_tags:
            row['tag_%s' % tag] = 1

        out.append(row)
        
    return pd.DataFrame(out)

In [ ]:
quotes_df = parse_quote_list(soup)
quotes_df

### Scraping quotes: `parse_author_info(author_soup)`
* Develop function on a single author page.
* Wrap code in `parse_author_info(author_soup)`

In [ ]:
url = quotes_df.auth_url.iloc[0]
author_resp = requests.get(url)
author_soup = bs4.BeautifulSoup(author_resp.text)

In [ ]:
# author bio, author bday, original url (to join)
author_soup

In [ ]:
dob = author_soup.find('span', attrs={'class': 'author-born-date'}).text
bio = author_soup.find('div', attrs={'class': 'author-description'}).text
author = author_soup.find('h3', attrs={'class': 'author-title'}).text

In [ ]:
auth_row = {'dob': dob, 'bio': bio, 'quote_auth': author}
auth_row # how to clean bio?

In [ ]:
def parse_author_info(author_soup):
    '''returns extracted author information from a parsed author page'''
    
    dob = author_soup.find('span', attrs={'class': 'author-born-date'}).text
    bio = author_soup.find('div', attrs={'class': 'author-description'}).text.strip()
    author = author_soup.find('h3', attrs={'class': 'author-title'}).text.strip()
    
    return {'dob': dob, 'bio': bio, 'quote_auth': author}

In [ ]:
parse_author_info(author_soup)

### Scraping quotes: Putting it together

* `scrape_pages` takes a list of urls and returns a dataframe of extracted info

In [ ]:
# proposed plan:

def scrape_pages(urllist):
    
    out = []
    for url in urllist:
        resp = requests.get(url)
        soup = bs4.BeautifulSoup(resp.text)
        if not resp.ok:
            return
        
        # parse quotes (dataframe):
        url_quotes = parse_quote_list(soup)
        out.append(url_quotes)
        
    quote_df = pd.concat(out, sort=False)
        
    # get author urls (fewest requests possible!)
    
    auth_out = []
    auth_urls = quote_df['auth_url'].unique()
    for auth_url in auth_urls:
        auth_resp = requests.get(auth_url)
        auth_soup = bs4.BeautifulSoup(auth_resp.text)
        auth = parse_author_info(auth_soup)
        auth_out.append(auth)
        
    auth_df = pd.DataFrame(auth_out)
    
    out_df = quote_df.merge(auth_df, on='quote_auth', how='left')
    
    return out_df

In [ ]:
urllist = ['http://quotes.toscrape.com/page/%d/' % x for x in range(1,3)]
urllist

In [ ]:
df = scrape_pages(urllist)

In [ ]:
df.head()

In [ ]:
df.quote_auth.value_counts()

In [ ]:
df.count()

### Scraping quotes: conclusion

* Make as few requests as possible
* Create a request/parsing plan *beforehand*
* Create your output schema *beforehand*
* Separate parsing and requests into different functions!

## Nested vs flat data structures

* Nested: HTML, JSON, XML
* Flat: CSV

Suppose we obtained the quotes data via an API and saved it to the file `quotes2scrape.json`
- `quotes2scrape.json` is a 'json records file'; each line is a valid json object

In [ ]:
import json
json.loads(open('data/quotes2scrape.json').readline())

In [ ]:
# read in all the lines: each element is a dictionary
L = [json.loads(x) for x in open('data/quotes2scrape.json')]

In [ ]:
# What happended to the tags column?
df = pd.DataFrame(L)
df.head()

In [ ]:
distinct_tags = np.unique(df.tags.sum())

In [ ]:
def list2series(taglist):
    return pd.Series({k:1 for k in taglist})

tags = df.tags.apply(list2series)
tags.head()

In [ ]:
# combine them
pd.concat([df, tags], axis=1)

### Converting JSON to CSV

* Flattening the nested list requires a lot of space. Why?
* We can't always read in *all* the JSON; might need to extract just what we need line-by-line.
* A JSON records file is **not** valid JSON. Why? Why can't we just use JSON?